# Sentiment Analysis Trial

In [1]:
# Import dependencies/modules
from textblob import TextBlob
import pandas as pd
import sys
import tweepy
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import time
import re
import string

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('vader_lexicon')
nltk.download('stopwords')
nltk.download('wordnet')

!pip install gensim
import gensim
from gensim.parsing.preprocessing import remove_stopwords 
import torch
import flair
from flair.models import TextClassifier
from flair.data import Sentence
from segtok.segmenter import split_single

from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from langdetect import detect
from sklearn.feature_extraction.text import CountVectorizer

import spacy

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/memme11/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/memme11/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Set column width to max 
pd.set_option('display.max_colwidth', None)

In [3]:
# Read in sample BTC data (Derived from kaggle - used to mimic or actual dataset which will be pulled from twitter using tweepy)
df = pd.read_csv('Cleaned_Bitcoin_tweets.csv')
df

,date,text,cleaned_text,user_followers,user_friends,user_favourites,hashtags,is_retweet
0,2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",False
1,2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",False
2,2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,9975.0,6.0,6.0,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",False
3,2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs",barclay s just wants to keep you safe this is why they let you spend your life savings on the lottery at the casino or trading stock futures but buying bitcoin on binance is a bridge to far ps they are a huge criminal money launderer,20668.0,451.0,0.0,['bitcoin'],False
4,2021-07-05 23:24:35,Why 'Cardano is just teasing us' - https://t.co/8JowtgVKvj $ADA #altcoin #news #bitcoin,why cardano is just teasing us ada altcoin news bitcoin,9276.0,175.0,0.0,"['altcoin', 'news', 'bitcoin']",False
...,...,...,...,...,...,...,...,...
48680,2021-02-05 11:00:24,"5⃣ #Blockchain 50 2021 by @DelRayMan, @Forbes , @ForbesCrypto ⬇️\n\n#cryptocurrency #bitcoin #crypto #blockchain… https://t.co/1g8ppqVtqA",5 blockchain 50 2021 by cryptocurrency bitcoin crypto blockchain,16813.0,818.0,1201.0,"['Blockchain', 'cryptocurrency', 'bitcoin', 'crypto', 'blockchain']",False
48681,2021-02-05 11:00:24,4⃣ 🎙️ Bloomberg LP #CryptoOutlook 2021 with @mikemcglone11 ⬇️\n\n#cryptocurrency #bitcoin #crypto #blockchain #btc… https://t.co/re2bksq9tW,4 bloomberg lp cryptooutlook 2021 with cryptocurrency bitcoin crypto blockchain btc,16813.0,818.0,1201.0,"['CryptoOutlook', 'cryptocurrency', 'bitcoin', 'crypto', 'blockchain', 'btc']",False
48682,2021-02-05 11:00:22,2⃣ Debunking 9 #Bitcoin Myths by @Patrick_Lowry_ ⬇️ \n\n#cryptocurrency #bitcoin #crypto #blockchain #btc… https://t.co/AYJQsuJ

In [4]:
# Draw a sample from the data 
tweets = df.head(500)

## Tokenizing & Reformatting Twitter Data

In [5]:
def get_wordnet_pos(word):
# Map POS tag to the first character lemmatize() accepts
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

# Function for tokenizing tweets (already cleaned using regex)
def second_clean(tweet, lemmatize = 'nltk'):
    tweet = remove_stopwords(tweet) # remove stopwords with Gensim

    lemmatizer = WordNetLemmatizer()
    tokenized = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(tweet)]
    
    # remove left over stop words with nltk
    tokenized = [token for token in tokenized if token not in stopwords.words("english")] 

    # remove non-alpha characters and keep the words of length >2 only
    tokenized = [token for token in tokenized if token.isalpha() and len(token)>2]

    return tokenized

# Function for joining tokenized list into one set of significant words 

def combine_tokens(tokenized): 
    non_tokenized = ' '.join([w for w in tokenized])
    return non_tokenized

In [6]:
# Execute function 
tweets['tokens'] = tweets['cleaned_text'].apply(lambda x: second_clean(x, 'nltk'))
tweets['final_clean'] = tweets['tokens'].apply(lambda x: combine_tokens(x))

/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/memme11/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
# Reorder tweets df
column_order = ['date','text', 'cleaned_text', 'tokens', 'final_clean', 'user_followers', 'user_friends', 'user_favourites', 'is_retweet', 'hashtags']
tweets = tweets[column_order]
tweets

,date,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags
0,2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']"
1,2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']"
2,2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']"
3,2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs",barclay s just wants to keep you safe this is why they let you spend your life sa

In [8]:
# Set the date as index and sort descending (i.e most recent tweets first)
tweets = tweets.set_index('date').sort_index(ascending=False)
tweets

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags
date,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']"
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']"
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']"
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs",barclay s just wants to keep you safe this is why they let you spend your life s

In [9]:
#Export to csv
output_file = 'Second_clean_twitter.csv'
tweets.to_csv(output_file, index='inplace')

# Sentiment Analysis

### Blob Sentiment Model

In [10]:
# Sentiment labels function 
def sentiment_labels(df, feature, value): 
    df.loc[df[value] > 0,feature] = 'positive'
    df.loc[df[value] == 0,feature] = 'neutral'
    df.loc[df[value] < 0,feature] = 'negative'

In [11]:
# Blob analysis function 
def blob_sentiment(df):
    df['blob_polarity'] = df['final_clean'].apply(lambda x: TextBlob(x).sentiment.polarity)
    
    df['blob_subjectivity'] = df['final_clean'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
    
    sentiment_labels(df, 'blob_sentiment', 'blob_polarity')
    
    return df[['final_clean', 'blob_polarity', 'blob_subjectivity', 'blob_sentiment']].head()
    

In [12]:
# Execute TextBlob Analysis and time
start = time.time()

blob_sentiment(tweets)

stop = time.time()

print(f'TextBlob analysis took: {round((stop-start)/60, 3)}minutes')

TextBlob analysis took: 0.011minutes


In [13]:
tweets.head(20)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,blob_polarity,blob_subjectivity,blob_sentiment
date,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.000000,0.000000,neutral
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",0.000000,0.000000,neutral
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",-0.191667,0.425000,negative
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge cri

In [14]:
# Blob Analysis Df
blob_df = pd.DataFrame(tweets)
blob_df.drop(columns=['text', 'tokens', 'user_followers', 'user_friends', 'user_favourites','is_retweet', 'hashtags'], inplace=True)
blob_df

,cleaned_text,final_clean,blob_polarity,blob_subjectivity,blob_sentiment
date,,,,,
2021-07-05 23:26:41,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,0.000000,0.000000,neutral
2021-07-05 23:26:36,lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,0.000000,0.000000,neutral
2021-07-05 23:25:27,lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,-0.191667,0.425000,negative
2021-07-05 23:25:19,barclay s just wants to keep you safe this is why they let you spend your life savings on the lottery at the casino or trading stock futures but buying bitcoin on binance is a bridge to far ps they are a huge criminal money launderer,barclay want safe let spend life saving lottery casino trading stock future buying bitcoin binance bridge far huge criminal money launderer,0.120000,0.615000,positive
2021-07-05 23:24:35,why cardano is just teasing us ada altcoin news bitcoin,cardano tease ada altcoin news bitcoin,0.000000,0.000000,neutral
...,...,...,...,...,...
2021-07-05 20:24:17,linkedin twitter giftidea facebook instagram tumblr socialmedia pinterest dogecoin reddit gift gifts giftideas shop shopping affiliatemarketing bitcoin shop gifts amp baskets,linkedin twitter giftidea facebook instagram tumblr socialmedia pinterest dogecoin reddit gift gift giftideas shop shopping affiliatemarketing bitcoin shop gift amp basket,0.000000,0.000000,neutral
2021-07-05 20:23:10,argentina vs colombia copa america semifinals live stream tv channel how to watch online news odds bitcoin sportsbook,argentina colombia copa america semifinal live stream channel watch online news odds bitcoin sportsbook,0.136364,0.500000,positive
2021-07-05 20:23:08,brazil vs peru copa america semifinal live stream tv channel how to watch online news odds bitcoin sportsbook,brazil peru copa america semifinal live stream channel watch online news odds bitcoin sportsbook,0.136364,0.500000,positive


### Vader Sentiment Model

In [15]:
# Vader sentiment analysis
# define function and variable for SentimentIntensityAnalyzer()

sia = SentimentIntensityAnalyzer()


def vader_sentiment(df, clean=True):
    
    # Create two options to compare tokenized clean vs regex clean
    if clean:
        target_col='final_clean'
        prefix = 'vader_clean_'
    else:
        target_col='cleaned_text'
        prefix='vader_'
        
    scores_col=prefix+'scores'
    
    compound_col = prefix+'polarity'
    
    sentiment = prefix+'sentiment'
    
    df[scores_col] = df[target_col].apply(lambda x:sia.polarity_scores(x))
    
    df[compound_col] = df[scores_col].apply(lambda d: d['compound'])
    
    sentiment_labels(df, sentiment, compound_col)

In [16]:
#Execute vader function
start = time.time()

vader_sentiment(tweets, clean=True)

stop = time.time()

print(f'Vader analysis took: {round((stop-start)/60, 3)}minutes')

Vader analysis took: 0.003minutes


In [17]:
tweets.head(20)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,blob_polarity,blob_subjectivity,blob_sentiment,vader_clean_scores,vader_clean_polarity,vader_clean_sentiment
date,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.000000,0.000000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",0.000000,0.000000,neutral,"{'neg': 0.0, 'neu': 0.961, 'pos': 0.039, 'compound': 0.0258}",0.0258,positive
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",-0.191667,0.425000,negative,"{'neg': 0.0, 'neu': 0.87, '

In [18]:
# Execute vader function on cleaned_text instead of final_clean (in order to compare scores)
vader_sentiment(tweets, clean=False)
tweets.head(20)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,blob_polarity,blob_subjectivity,blob_sentiment,vader_clean_scores,vader_clean_polarity,vader_clean_sentiment,vader_scores,vader_polarity,vader_sentiment
date,,,,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.000000,0.000000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",0.000000,0.000000,neutral,"{'neg': 0.0, 'neu': 0.961, 'pos': 0.039, 'compound': 0.0258}",0.0258,positive,"{'neg': 0.0, 'neu': 0.974, 'pos': 0.026, 'compound': 0.0258}",0.0258,positive
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity me

In [19]:
# Calculate accuracy score of vader analysis on regex cleaned text vs tokenized text
from sklearn.metrics import accuracy_score

accuracy_score(tweets['vader_sentiment'], tweets['vader_clean_sentiment'])

0.956

In [20]:
# Vader df 
vader_columns = ['text', 'cleaned_text', 'tokens', 'final_clean', 'user_followers', 'user_friends', 'user_favourites','is_retweet', 'hashtags', 'vader_clean_scores', 'vader_clean_polarity', 'vader_clean_sentiment', 'vader_scores', 'vader_polarity', 'vader_sentiment']
vader_df = pd.DataFrame(tweets[vader_columns])
vader_df.head(20)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,vader_clean_scores,vader_clean_polarity,vader_clean_sentiment,vader_scores,vader_polarity,vader_sentiment
date,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']","{'neg': 0.0, 'neu': 0.961, 'pos': 0.039, 'compound': 0.0258}",0.0258,positive,"{'neg': 0.0, 'neu': 0.974, 'pos': 0.026, 'compound': 0.0258}",0.0258,positive
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNe

### Flair Sentiment Model

In [21]:
#Flair model 

flair_classifier = TextClassifier.load('en-sentiment')
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2021-07-26 18:03:09,347 loading file /Users/memme11/.flair/models/sentiment-en-mix-distillbert_4.pt
2021-07-26 18:03:14,286 loading file /Users/memme11/.flair/models/sentiment-en-mix-distillbert_4.pt


In [22]:
#Flair functions 

# Function for splitting text into sentences
def flair_sentences(regex_text):
    sentences = [sentence for sentence in split_single(regex_text)]
    return sentences

# Function that creates numerical scores for sentiment analysis - scores fall between -1 and 1
def flair_predict_sentences(sentence):
    if sentence == "":
        return 0 
    
    text=Sentence(sentence)
    
    flair_classifier.predict(text)
    value = text.labels[0].to_dict()['value']
    
    if value == 'POSITIVE':
        result = text.to_dict()['labels'][0]['confidence']
        
    else: 
        result = -(text.to_dict()['labels'][0]['confidence'])
        
    return round(result, 2)

# Function that executes the flair_predict_sentences function and adds it to the df
def flair_sentence_scores(sentences):
    results=[]
    
    for i in range(0, len(sentences)):
        results.append(flair_predict_sentences(sentences[i]))
        results.append(flair_predict_sentences(sentences[0]))
        
    return results

# Sums the scores and rounds by 2 digits
def flair_sum(scores):
    result = round(sum(scores), 2)
    return result

# Averages the added scores and rounds
def avg_scores(scores):
    result = round(np.mean(scores), 2)
    return result

# Sentiment score for full tweets
def flair_tweet_score(tokenized_text):
    if not tokenized_text:
        return 0
    
    tweet = Sentence(tokenized_text)
    flair_classifier.predict(tweet)
    value = tweet.labels[0].to_dict()['value']
    
    if value == 'POSITIVE':
        result = tweet.to_dict()['labels'][0]['confidence']
    
    else: 
        result = -(tweet.to_dict()['labels'][0]['confidence'])
        
    return round(result, 2)

# Determining sentiment labels based on scores
def flair_sentiment(polarity):
    if polarity > 0.6:
        return 'positive'
    
    if polarity < -0.6:
        return 'negative'
    
    else:
        return 'neutral'

In [23]:
# Flair model assessing per sentence 
# divide tweet into sentences, score them, then get avg scores of each sentence
# Calling some of the above functions and timing

start = time.time()

tweets['sentences'] = tweets['text'].apply(flair_sentences)
tweets['flair_scores'] = tweets['sentences'].apply(flair_sentence_scores)
tweets['flair_avg_score'] = tweets['flair_scores'].apply(avg_scores)

end = time.time()
print(round((stop-start)/60, 3))

-0.166


In [24]:
tweets.head(20)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,blob_polarity,...,blob_sentiment,vader_clean_scores,vader_clean_polarity,vader_clean_sentiment,vader_scores,vader_polarity,vader_sentiment,sentences,flair_scores,flair_avg_score
date,,,,,,,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.000000,...,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,"[#Thailand #Boutiquehotels For Sale!, #willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement , #thailandrealestate #phuket #bkk, #bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz]","[1.0, 1.0, 1.0, 1.0, 0.99, 1.0, 0.88, 1.0]",0.98
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",0.000000,...,neutral,"{'neg': 0.0, 'neu': 0.961, 'pos': 0.039, 'compound': 0.0258}",0.0258,positive,"{'neg': 0.0, 'neu': 0.974, 'pos': 0.026, 'compound': 0.0258}",0.0258,positive,"[LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins., (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews]","[-1.0, -1.0, -1.0, -1.0]",-1.00
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observe

In [25]:
# Flair sentiment assessing per tweet
tweets['flair_cleaned_text'] = tweets['cleaned_text'].apply(flair_tweet_score)
tweets['flair_token'] = tweets['final_clean'].apply(flair_tweet_score)

tweets.head(30)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,blob_polarity,...,vader_clean_polarity,vader_clean_sentiment,vader_scores,vader_polarity,vader_sentiment,sentences,flair_scores,flair_avg_score,flair_cleaned_text,flair_token
date,,,,,,,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.000000,...,0.0000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,"[#Thailand #Boutiquehotels For Sale!, #willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement , #thailandrealestate #phuket #bkk, #bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz]","[1.0, 1.0, 1.0, 1.0, 0.99, 1.0, 0.88, 1.0]",0.98,1.00,1.00
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",0.000000,...,0.0258,positive,"{'neg': 0.0, 'neu': 0.974, 'pos': 0.026, 'compound': 0.0258}",0.0258,positive,"[LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins., (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews]","[-1.0, -1.0, -1.0, -1.0]",-1.00,-1.00,-0.85
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node

In [26]:
# Use sentiment function to determine sentiment of regex cleaned text, and sentiment of avg scores from sentences
tweets['cleaned_sentiment'] = tweets['flair_token'].apply(flair_sentiment)
tweets['uncleaned_sentiment'] = tweets['flair_cleaned_text'].apply(flair_sentiment)
tweets['avg_sentiment'] = tweets['flair_avg_score'].apply(flair_sentiment)

tweets.head(30)

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,blob_polarity,...,vader_polarity,vader_sentiment,sentences,flair_scores,flair_avg_score,flair_cleaned_text,flair_token,cleaned_sentiment,uncleaned_sentiment,avg_sentiment
date,,,,,,,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.000000,...,0.0000,neutral,"[#Thailand #Boutiquehotels For Sale!, #willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement , #thailandrealestate #phuket #bkk, #bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz]","[1.0, 1.0, 1.0, 1.0, 0.99, 1.0, 0.88, 1.0]",0.98,1.00,1.00,positive,positive,positive
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",0.000000,...,0.0258,positive,"[LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins., (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews]","[-1.0, -1.0, -1.0, -1.0]",-1.00,-1.00,-0.85,negative,negative,negative
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Light

In [27]:
# Reorder columns and create flair df 
flair_columns = ['text', 'cleaned_text', 'final_clean', 'user_followers', 'user_friends', 'user_favourites','is_retweet', 'hashtags', 'flair_avg_score', 'flair_cleaned_text', 'flair_token', 'cleaned_sentiment', 'uncleaned_sentiment', 'avg_sentiment']

flair_df = pd.DataFrame(tweets[flair_columns])
flair_df

,text,cleaned_text,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,flair_avg_score,flair_cleaned_text,flair_token,cleaned_sentiment,uncleaned_sentiment,avg_sentiment
date,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']",0.98,1.00,1.00,positive,positive,positive
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']",-1.00,-1.00,-0.85,negative,negative,negative
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNetwork', 'Lightning', 'LN', 'bitcoin']",0.80,0.53,-0.79,negative,neutral,positive
2021-07-05 23:25:19,"Barclay's just wants to keep you safe, this is why they let you spend your life savings on the lottery, at the casino, or trading stock futures. But buying #bitcoin on Binance is a bridge to far. \n\nPS they are a huge criminal money launderer\n\nhttps://t.co/FUb9BMnavs",barclay s just wants to keep you safe this is why they let you spend your life savings on the lottery at the casino or trading stock futures but buying bitcoin on binance is a bridge to far ps they are a huge criminal money launderer,barclay want safe let spend life saving lottery casino trading stock future buying bitcoin binance bridge far huge criminal money launderer,20668.0,451.0,0.0,False,['bitcoin'],-0.91,-1.00,-1.00,negative,negative,negative
2021-07-05 23:24:35,Why 'Cardano is just teasing us' - https://t.co/8J

In [28]:
vader_df

,text,cleaned_text,tokens,final_clean,user_followers,user_friends,user_favourites,is_retweet,hashtags,vader_clean_scores,vader_clean_polarity,vader_clean_sentiment,vader_scores,vader_polarity,vader_sentiment
date,,,,,,,,,,,,,,,
2021-07-05 23:26:41,#Thailand #Boutiquehotels For Sale! \n#willroadhouse #AoyRoadhouse #compassgroupinternational #ThailandBeachfrontHomes #ThailandBeachfrontHome #retirement \n#thailandrealestate #phuket #bkk\n#bangkok #huahin #jomtienbeach #jomtien #pattaya #thailandexpats #bitcoin #cryptocurrency https://t.co/GbFTpmaesz,thailand boutiquehotels for sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,"[thailand, boutiquehotels, sale, willroadhouse, aoyroadhouse, compassgroupinternational, thailandbeachfronthomes, thailandbeachfronthome, retirement, thailandrealestate, phuket, bkk, bangkok, huahin, jomtienbeach, jomtien, pattaya, thailandexpats, bitcoin, cryptocurrency]",thailand boutiquehotels sale willroadhouse aoyroadhouse compassgroupinternational thailandbeachfronthomes thailandbeachfronthome retirement thailandrealestate phuket bkk bangkok huahin jomtienbeach jomtien pattaya thailandexpats bitcoin cryptocurrency,9150.0,128.0,47.0,False,"['Thailand', 'Boutiquehotels', 'willroadhouse', 'AoyRoadhouse', 'compassgroupinternational', 'ThailandBeachfrontHomes', 'ThailandBeachfrontHome', 'retirement', 'thailandrealestate', 'phuket', 'bkk', 'bangkok', 'huahin', 'jomtienbeach', 'jomtien', 'pattaya', 'thailandexpats', 'bitcoin', 'cryptocurrency']","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.0000,neutral
2021-07-05 23:26:36,"LUNALAND ($LLN) will burn coin supply until we reach 1 Lunar distance in coins. (400M coins) A burn rate of 99.96% from the initial pool of 1 Trillion coins reduces coin supply, #presale #blockchain #cryptotrading #investing #cryptocurrency #bitcoin #trading #altcoin #cryptonews",lunaland lln will burn coin supply until we reach 1 lunar distance in coins 400m coins a burn rate of 99 96 from the initial pool of 1 trillion coins reduces coin supply presale blockchain cryptotrading investing cryptocurrency bitcoin trading altcoin cryptonews,"[lunaland, lln, burn, coin, supply, reach, lunar, distance, coin, coin, burn, rate, initial, pool, trillion, coin, reduces, coin, supply, presale, blockchain, cryptotrading, invest, cryptocurrency, bitcoin, trading, altcoin, cryptonews]",lunaland lln burn coin supply reach lunar distance coin coin burn rate initial pool trillion coin reduces coin supply presale blockchain cryptotrading invest cryptocurrency bitcoin trading altcoin cryptonews,10155.0,7181.0,8833.0,False,"['presale', 'blockchain', 'cryptotrading', 'investing', 'cryptocurrency', 'bitcoin', 'trading', 'altcoin', 'cryptonews']","{'neg': 0.0, 'neu': 0.961, 'pos': 0.039, 'compound': 0.0258}",0.0258,positive,"{'neg': 0.0, 'neu': 0.974, 'pos': 0.026, 'compound': 0.0258}",0.0258,positive
2021-07-05 23:25:27,"#LightningNetwork \n\nhttps://t.co/Q0CaYRhLWD observed:\n22,365 nodes (12,502 with active channels)\n53,035 channels\n1,682.728 BTC capacity ($57,335,747)\n\nmedian node capacity: 0.002 BTC ($74.64)\n\npast 24h:\n+72 nodes +972 channels +17.078 BTC ($581,892)\n\n#Lightning #LN #bitcoin $BTC",lightningnetwork observed 22 365 nodes 12 502 with active channels 53 035 channels 1 682 728 btc capacity 57 335 747 median node capacity 0 002 btc 74 64 past 24h 72 nodes 972 channels 17 078 btc 581 892 lightning ln bitcoin btc,"[lightningnetwork, observe, node, active, channel, channel, btc, capacity, median, node, capacity, btc, past, node, channel, btc, lightning, bitcoin, btc]",lightningnetwork observe node active channel channel btc capacity median node capacity btc past node channel btc lightning bitcoin btc,9975.0,6.0,6.0,False,"['LightningNe

# Comparing Sentiment Model Scores 

In [29]:
blob_df['blob_sentiment'].value_counts()

neutral     313
positive    126
negative     61
Name: blob_sentiment, dtype: int64

In [30]:
vader_df['vader_sentiment'].value_counts()

positive    297
neutral     104
negative     99
Name: vader_sentiment, dtype: int64

In [31]:
flair_df['cleaned_sentiment'].value_counts()

positive    243
negative    236
neutral      21
Name: cleaned_sentiment, dtype: int64

In [32]:
flair_df['uncleaned_sentiment'].value_counts()

positive    258
negative    230
neutral      12
Name: uncleaned_sentiment, dtype: int64

In [33]:
flair_df['avg_sentiment'].value_counts()

positive    217
neutral     157
negative    126
Name: avg_sentiment, dtype: int64

In [52]:
# Function for determining avg sentiment scores for each model
def pos_neg_neutral_avg(df):
    
    positive = []
    neutral = []
    negative = []
    
    for values in df:
        if values > 0:
            positive.append(values)
        
        elif values < 0:
            negative.append(values)
        
        else:
            neutral.append(values)
    
    print(f'Positive score average for {df.name} = {round(np.mean(positive), 2)}')
    print(f'Neutral score average for {df.name} = {round(np.mean(neutral), 2)}')
    print(f'Negative score average for {df.name} = {round(np.mean(negative), 2)}')
    
    print(f'Overall crypto sentiment score is = {round(np.mean(df), 4)}')

In [53]:
# Create variables for each model's polarity column
textblob_values = blob_df.loc[:, 'blob_polarity']
vader_values = vader_df.loc[:, 'vader_polarity']
flair_values = flair_df.loc[:, 'flair_token']

In [54]:
# Average scores for TextBlob Analysis
pos_neg_neutral_avg(textblob_values)

Positive score average for blob_polarity = 0.32
Neutral score average for blob_polarity = 0.0
Negative score average for blob_polarity = -0.15
Overall crypto sentiment score is = 0.0623


In [55]:
# Average scores for Vader Analysis
pos_neg_neutral_avg(vader_values)

Positive score average for vader_polarity = 0.52
Neutral score average for vader_polarity = 0.0
Negative score average for vader_polarity = -0.39
Overall crypto sentiment score is = 0.2309


In [56]:
# Average scores for Flair Analysis
pos_neg_neutral_avg(flair_values)

Positive score average for flair_token = 0.92
Neutral score average for flair_token = nan
Negative score average for flair_token = -0.93
Overall crypto sentiment score is = -0.0006
